# 0. Attaching libraries

In [44]:
import os
import sqlite3
import pandas as pd
import sys
#from ydata_profiling import ProfileReport # for profiling
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
sns.set_style("whitegrid")       # optional aesthetics
%matplotlib inline 

In [45]:
from dateutil.relativedelta import relativedelta
import builtins

In [46]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn")

# 1. Reading the core Dataset from the golden source

In [42]:
csv_file  = "https://raw.githubusercontent.com/mithridata-com/NOVAIMS_BDMwDS_PROJECT/refs/heads/main/00%20Data/Dataset.csv?token=GHSAT0AAAAAADEUY5L67SA3ZFHKQZRP2KIM2CHCNUA"

excel_path  = (r"C:\Users\dimet\Documents\GitHub\NOVAIMS_BDMwDS_PROJECT\02 Output\02 TB Conversion Analytics.xlsx")

In [ ]:
# ── Load the CSV into a DataFrame ──────────────────────────────────────────
df = pd.read_csv(csv_file, low_memory=False)

In [6]:
# Quick sanity check (optional)
print(df.shape)       # prints (rows, columns)

(9373, 78)


## NEW COLUMNS

In [7]:
df["customer_id"] = df[["zipcode_link", "zip4", "place_residence","birth_date", "gender"]].astype(str).agg("-".join, axis=1)

In [10]:
# 1. Build boolean conditions based on == "Y"
conditions = [
    df['wa'] == 'Y',
    df['wa_bep_ca'] == 'Y',
    df['wa_ca'] == 'Y'
]

# 2. Corresponding labels
choices = ['1. Only liability insurance', '2. Liability + limited casco', '3. Liability + full casco']

# 3. Create the new column, defaulting to NaN if none of the three has "Y"
df['coverage_type'] = np.select(conditions, choices, default='')

In [11]:
# ── 1. External “today” reference ───────────────────────────────────────────
current_date = pd.to_datetime("2019-11-01")    # ← use any value you like
fancy_date = current_date.strftime("%B %d %Y")

In [12]:
# ── 2. Parse birth_date & compute age ───────────────────────────────────────
# 2-a. Ensure birth_date is a datetime column
df["birth_date"] = pd.to_datetime(df["birth_date"], errors="coerce")

# 2-b. Convert to integer years using dateutil.relativedelta for accuracy
df["age"] = df["birth_date"].apply(
    lambda bd: relativedelta(current_date, bd).years if pd.notnull(bd) else pd.NA
)
# ── 1. Ensure age is numeric ────────────────────────────────────────────────
# (errors='coerce' converts any bad strings to NaN so they drop out later)
df["age"] = pd.to_numeric(df["age"], errors="coerce")

# ── 2. Create age bands ─────────────────────────────────────────────────────
age_bins   = [0, 25, 35, 45, 55, 65, 120]               # tweak if needed
age_labels = ["<25", "25–34", "35–44", "45–54", "55–64", "65+"]

In [13]:
df["age_band"] = pd.cut(df["age"],
                           bins=age_bins,
                           labels=age_labels,
                           right=False)

In [14]:
df["urb_norm"] = (
    df["URB"]
      .astype(str)
      .str.strip()
      .str[0]                    # first letter
      .str.upper()
      .replace({"1": "1-VeryHigh", "2": "2-High", "3": "3-Mid-to-High", "4": "4-Mid", "5": "5-Mid-to-Low", "6": "6-Low","7": "7-VeryLow", "0": "Unknown", "N" : "Unknown"})
)

In [15]:
month_map = {
    "January":   "01",
    "February":  "02",
    "March":     "03",
    "April":     "04",
    "May":       "05",
    "June":      "06",
    "July":      "07",
    "August":    "08",
    "September": "09",
    "October":   "10",
    "November":  "11",
    "December":  "12"
}

df["buildmonth_num"] = df["buildmonth_car"].map(month_map)

In [16]:
# 2. Combine year + month + “01” (first of month) into a YYYY-MM-DD string and convert to datetime:
df["build_date"] = pd.to_datetime(
    df["buildyear_car"].astype(str).str[:4] + "-" + 
    df["buildmonth_num"] + "-01"
)

# Now 'build_date' holds a Timestamp for the first day of that month/year.
print(df[["buildyear_car", "buildmonth_car", "build_date"]].head())

   buildyear_car buildmonth_car build_date
0         2016.0           June 2016-06-01
1         2015.0           June 2015-06-01
2         2016.0          April 2016-04-01
3         2009.0           June 2009-06-01
4         2003.0        January 2003-01-01


In [17]:
# Calculate car_age
df['car_age'] = current_date.year - df['build_date'].dt.year - (
    (current_date.month < df['build_date'].dt.month) |
    ((current_date.month == df['build_date'].dt.month) & (current_date.day < df['build_date'].dt.day))
).astype(int)

In [18]:
# 1) Make sure `policy_start_date` is a datetime:
df['policy_start_date'] = pd.to_datetime(
    df['policy_start_date'],
    errors='coerce'
)

In [19]:
# Define age bins and labels
bins = [0, 3, 7, 11, 100]
labels = ['0-3', '4-7', '8-11', '12+']

# Create normalized age categories
df['car_age_norm'] = pd.cut(df['car_age'], bins=bins, labels=labels, include_lowest=True).cat.add_categories('Unknown').fillna("Unknown")


In [20]:
df

,affinity_name,status_report,offer_number,policy_number,zipcode_link,zip4,birth_date,brand,date_offer,date_request,...,churn,customer_id,coverage_type,age,age_band,urb_norm,buildmonth_num,build_date,car_age,car_age_norm
0,Insuro,Requestwithdrawn,1000,10000.0,10000,2132,1985-01-01,HYUNDAI,2018-10-11,2018-10-11,...,1,10000-2132-nan-1985-01-01-nan,1. Only liability insurance,34.0,25–34,3-Mid-to-High,06,2016-06-01,3.0,0-3
1,other,Tailoredofferwithdrawn,1001,NaN,10001,6027,1987-04-01,AUDI,2018-10-11,NaN,...,-1,10001-6027-nan-1987-04-01-nan,,32.0,25–34,7-VeryLow,06,2015-06-01,4.0,4-7
2,other,Incompleterequest,1002,NaN,10002,3824,1972-11-01,VOLKSWAGEN,2018-10-11,NaN,...,-1,10002-3824-nan-1972-11-01-nan,3. Liability + full casco,47.0,45–54,2-High,04,2016-04-01,3.0,0-3
3,other,Policycreated,1003,10002.0,10003,6921,1983-08-01,MAZDA,2018-10-11,2018-10-11,...,1,10003-6921-nan-1983-08-01-nan,2. Liability + limited casco,36.0,35–44,4-Mid,06,2009-06-01,10.0,8-11
4,other,Policycreated,1004,10003.0,10004,8266,1990-04-01,VOLVO,2018-10-12,2018-10-12,...,1,10004-8266-nan-1990-04-01-nan,1. Only liability insurance,29.0,25–34,4-Mid,01,2003-01-01,16.0,12+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9368,other,Requestaccepted,9439,11946.0,15568,1161,1960-09-01,VOLKSWAGEN,2020-03-08,2020-03-08,...,-1,15568-1161-nan-1960-09-01-nan,3. Liability + full casco,59.0,55–64,6-Low,12,2011-12-01,7.0,4-7
9369,other,Waitforapproval,9440,11947.0,15582,5015,1953-04-01,NISSAN,2020-03-08,2020-03-08,...,-1,15582-5015-nan-1953-04-01-nan,3. Liability + full casco,66.0,65+,2-High,02,2017-02-01,2.0,0-3
9370,Insuro,Tailoredofferrequested,9441,NaN,10332,3078,1976-04-01,TOYOTA,2020-03-08,NaN,...,-1,10332-3078-nan-1976-04-01-nan,2. Liability + limited casco,43.0,35–44,1-VeryHigh,01,2004-01-01,15.0,12+
9371,T&B,Calculatenewpremium,9442,NaN,12968,1965,1951-07-01,TOYOTA,2020-03-08,NaN,...,-1,12968-1965-nan-1951-07-01-nan,2. Liability + limited casco,68.0,65+,4-Mid,09,2004-09-01,15.0,12+


## ANALYSIS

In [21]:
# Create a pivot table counting policy_number for each combination of the three columns
conv_table = df.pivot_table(
    index=['affinity_name'],
    #columns=['churn'],
    values=['conv',"policy_number","offer_number"],
    aggfunc={
            "conv": "mean",
            "policy_number": lambda x: len(x.unique()),
            "offer_number": lambda x: len(x.unique())
        }
).reset_index()

conv_table

,affinity_name,conv,offer_number,policy_number
0,Insuro,0.254237,1305,401
1,Seguros International Ltd.,0.250155,1434,448
2,T&B,0.133244,3398,593
3,other,0.165588,2345,509


In [22]:
conv_table["ratio"] = conv_table["policy_number"]/conv_table["offer_number"]

In [23]:
conv_table

,affinity_name,conv,offer_number,policy_number,ratio
0,Insuro,0.254237,1305,401,0.307280
1,Seguros International Ltd.,0.250155,1434,448,0.312413
2,T&B,0.133244,3398,593,0.174514
3,other,0.165588,2345,509,0.217058


In [24]:
# 2) Group by `customer_id` and take the minimum `policy_start_date`
first_policy = (
    df
    .groupby('policy_number', as_index=False)['policy_start_date']
    .min()
    .rename(columns={'policy_start_date': 'first_policy_date'})
)

# 3) From that min date, extract the quarter (e.g. “2021Q1”, “2021Q2”…)
#    We’ll use pandas’ Period functionality for a clean “YYYYQ#” string.
first_policy['cohort'] = first_policy['first_policy_date'].dt.to_period('m').astype(str)  

C:\Users\dimet\AppData\Local\Temp\ipykernel_34452\4794884.py:11: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'M' instead.
  first_policy['cohort'] = first_policy['first_policy_date'].dt.to_period('m').astype(str)


In [25]:
df = df.merge(
    first_policy[['policy_number', 'cohort']],
    on='policy_number',
    how='left'
)

In [26]:
# Create a pivot table counting policy_number for each combination of the three columns
product_df = df.pivot_table(
    index=['coverage_type','affinity_name'],
    #columns=['affinity_name'],
    values=['conv'],
    aggfunc={
            "conv": "mean"
        },
    margins=True,
    margins_name="Total"
    
).reset_index().rename(columns={'policy_number': 'count_policy_number'})

product_df

,coverage_type,affinity_name,conv
0,,Insuro,0.000000
1,,Seguros International Ltd.,0.000000
2,,T&B,0.000000
3,,other,0.000000
4,1. Only liability insurance,Insuro,0.345083
5,1. Only liability insurance,Seguros International Ltd.,0.302245
6,1. Only liability insurance,T&B,0.258649
7,1. Only liability insurance,other,0.214559
8,2. Liability + limited casco,Insuro,0.225316
9,2. Liability + limited casco,Seguros International Ltd.,0.243952


In [27]:
# Create a pivot table counting policy_number for each combination of the three columns
urban_df = df.pivot_table(
    index=["urb_norm","affinity_name"],
    #columns=['affinity_name'],
    values=['conv'],
    aggfunc="mean",
    margins=True,
    margins_name="Total"
    
).reset_index().rename(columns={'policy_number': 'count_policy_number'})

urban_df

,urb_norm,affinity_name,conv
0,1-VeryHigh,Insuro,0.254967
1,1-VeryHigh,Seguros International Ltd.,0.277603
2,1-VeryHigh,T&B,0.238182
3,1-VeryHigh,other,0.164474
4,2-High,Insuro,0.254355
5,2-High,Seguros International Ltd.,0.259740
6,2-High,T&B,0.121160
7,2-High,other,0.173653
8,3-Mid-to-High,Insuro,0.259091
9,3-Mid-to-High,Seguros International Ltd.,0.283088


In [28]:
# 1. Define the mapping from integer code → descriptive label
stage_map = {
    0: "unknown",
    1: "young singles",
    2: "adult singles",
    3: "older singles",
    4: "families with young children",
    6: "families with older children",
    7: "young couples without children",
    8: "adult couples without children",
    9: "older couples without children"
    # Note: code 5 is not defined here; any 5s will become NaN unless you add a mapping.
}

# 2. Apply that mapping to create a new column
df["STAGE_OF_LIFE_label"] = df["STAGE_OF_LIFE"].map(stage_map)

# 4. Quick check
print(df[["STAGE_OF_LIFE", "STAGE_OF_LIFE_label"]].head())

   STAGE_OF_LIFE             STAGE_OF_LIFE_label
0            NaN                             NaN
1            9.0  older couples without children
2            4.0    families with young children
3            4.0    families with young children
4            4.0    families with young children


In [29]:
# Create a pivot table counting policy_number for each combination of the three columns
stage_df = df.pivot_table(
    index=["STAGE_OF_LIFE","STAGE_OF_LIFE_label",'affinity_name'],
    #columns=['affinity_name'],
    values=['conv'],
    aggfunc="mean",
    margins=True,
    margins_name="Total"
    
).reset_index()

stage_df

,STAGE_OF_LIFE,STAGE_OF_LIFE_label,affinity_name,conv
0,1.0,young singles,Insuro,0.198718
1,1.0,young singles,Seguros International Ltd.,0.302469
2,1.0,young singles,T&B,0.200000
3,1.0,young singles,other,0.186813
4,2.0,adult singles,Insuro,0.324786
5,2.0,adult singles,Seguros International Ltd.,0.290323
6,2.0,adult singles,T&B,0.177057
7,2.0,adult singles,other,0.213115
8,3.0,older singles,Insuro,0.237762
9,3.0,older singles,Seguros International Ltd.,0.261905


In [30]:
# Create a pivot table counting policy_number for each combination of the three columns
income_df = df.pivot_table(
    index=["INCOME",'affinity_name'],
    #columns=['affinity_name'],
    values=['conv'],
    aggfunc="mean",
    margins=True,
    margins_name="Total"
    
).reset_index()

income_df

,INCOME,affinity_name,conv
0,1.0,Insuro,0.172840
1,1.0,Seguros International Ltd.,0.200000
2,1.0,T&B,0.123675
3,1.0,other,0.137529
4,2.0,Insuro,0.220455
5,2.0,Seguros International Ltd.,0.236542
6,2.0,T&B,0.115834
7,2.0,other,0.172798
8,3.0,Insuro,0.305825
9,3.0,Seguros International Ltd.,0.269444


In [31]:
residence_df = (
    df
    .pivot_table(index=["PROVINCE","affinity_name"],
                 #columns=["affinity_name"],
                 values=["conv"],          # any non-null column works
                 aggfunc="mean",
                 margins=True,
                 margins_name="Total")
    .reset_index()
    #.sort_values(by="num_customers", ascending=False)
)

residence_df

,PROVINCE,affinity_name,conv
0,Drenthe,Insuro,0.250000
1,Drenthe,Seguros International Ltd.,0.173913
2,Drenthe,T&B,0.066667
3,Drenthe,other,0.117647
4,Flevoland,Insuro,0.320755
5,Flevoland,Seguros International Ltd.,0.215385
6,Flevoland,T&B,0.137931
7,Flevoland,other,0.177778
8,Friesland,Insuro,0.090909
9,Friesland,Seguros International Ltd.,0.156250


In [32]:
# Create a pivot table counting policy_number for each combination of the three columns
brand_df = df.pivot_table(
    index=["brand","affinity_name"],
    #columns=['affinity_name'],
    values=['conv'],
    aggfunc="mean",
    margins=True,
    margins_name="Total"
    
).reset_index()

brand_df

,brand,affinity_name,conv
0,ALFA ROMEO,Insuro,0.500000
1,ALFA ROMEO,Seguros International Ltd.,0.333333
2,ALFA ROMEO,T&B,0.210526
3,ALFA ROMEO,other,0.133333
4,APRILIA,Insuro,0.000000
...,...,...,...
182,VOLVO,other,0.179641
183,YAMAHA,Insuro,0.066667
184,YAMAHA,T&B,0.000000
185,YAMAHA,other,0.076923


In [33]:
# Create a pivot table counting policy_number for each combination of the three columns
age_df = df.pivot_table(
    index=["age_band",'affinity_name'],
    #columns=['affinity_name'],
    values=['conv'],
    aggfunc="mean",
    margins=True,
    margins_name="Total"
    
).reset_index()

age_df

C:\Users\dimet\AppData\Local\Temp\ipykernel_34452\782192441.py:2: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  age_df = df.pivot_table(


,age_band,affinity_name,conv
0,<25,Insuro,0.256303
1,<25,Seguros International Ltd.,0.301724
2,<25,T&B,0.212806
3,<25,other,0.178378
4,25–34,Insuro,0.289256
5,25–34,Seguros International Ltd.,0.312605
6,25–34,T&B,0.228464
7,25–34,other,0.247449
8,35–44,Insuro,0.292135
9,35–44,Seguros International Ltd.,0.238095


In [34]:
loan_df = df.pivot_table(
    index=["LOAN","affinity_name"],
    values=['conv'],
    aggfunc="mean",
    margins=True,
    margins_name="Total"
    
).reset_index()

loan_df

,LOAN,affinity_name,conv
0,1.0,Insuro,0.244444
1,1.0,Seguros International Ltd.,0.198381
2,1.0,T&B,0.072197
3,1.0,other,0.153430
4,2.0,Insuro,0.229787
5,2.0,Seguros International Ltd.,0.230303
6,2.0,T&B,0.125828
7,2.0,other,0.143750
8,3.0,Insuro,0.269625
9,3.0,Seguros International Ltd.,0.259947


In [35]:
savings_df = df.pivot_table(
    index=["SAVINGS","affinity_name"],
    #columns=['affinity_name'],
    values=['conv'],
    aggfunc="mean",
    margins=True,
    margins_name="Total"
).reset_index()

savings_df

,SAVINGS,affinity_name,conv
0,1.0,Insuro,0.428571
1,1.0,Seguros International Ltd.,0.363636
2,1.0,T&B,0.128205
3,1.0,other,0.111111
4,2.0,Insuro,0.229167
5,2.0,Seguros International Ltd.,0.267857
6,2.0,T&B,0.152882
7,2.0,other,0.132530
8,3.0,Insuro,0.254839
9,3.0,Seguros International Ltd.,0.279188


In [36]:
car_df = df.pivot_table(
    index=["CAR","affinity_name"],
    #columns=['affinity_name'],
    values=['conv'],
    aggfunc="mean",
    margins=True,
    margins_name="Total"
    
).reset_index()

car_df

,CAR,affinity_name,conv
0,1.0,Insuro,0.288591
1,1.0,Seguros International Ltd.,0.291005
2,1.0,T&B,0.213942
3,1.0,other,0.172932
4,2.0,Insuro,0.260563
5,2.0,Seguros International Ltd.,0.316176
6,2.0,T&B,0.177184
7,2.0,other,0.182741
8,3.0,Insuro,0.296296
9,3.0,Seguros International Ltd.,0.264865


In [37]:
own_house_df = df.pivot_table(
    index=["OWN_HOUSE","affinity_name"],
    #columns=['affinity_name'],
    values=['conv'],
    aggfunc="mean",
    margins=True,
    margins_name="Total"
    
).reset_index()

own_house_df

,OWN_HOUSE,affinity_name,conv
0,1.0,Insuro,0.248000
1,1.0,Seguros International Ltd.,0.290456
2,1.0,T&B,0.175853
3,1.0,other,0.144928
4,2.0,Insuro,0.269565
5,2.0,Seguros International Ltd.,0.267206
6,2.0,T&B,0.167874
7,2.0,other,0.182768
8,3.0,Insuro,0.271429
9,3.0,Seguros International Ltd.,0.273973


In [38]:
car_age_df = df.pivot_table(
    index=["car_age_norm","affinity_name"],
    #columns=['affinity_name'],
    values=['conv'],
    aggfunc="mean",
    margins=True,
    margins_name="Total"
    
).reset_index()

car_age_df

C:\Users\dimet\AppData\Local\Temp\ipykernel_34452\868622937.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  car_age_df = df.pivot_table(


,car_age_norm,affinity_name,conv
0,0-3,Insuro,0.201389
1,0-3,Seguros International Ltd.,0.185185
2,0-3,T&B,0.056931
3,0-3,other,0.156197
4,4-7,Insuro,0.271375
5,4-7,Seguros International Ltd.,0.297376
6,4-7,T&B,0.135854
7,4-7,other,0.172414
8,8-11,Insuro,0.255814
9,8-11,Seguros International Ltd.,0.296399


In [39]:
status_df = df.pivot_table(
    index=["status_report","affinity_name"],
    #columns="affinity_name",
    values=['offer_number','policy_number'],
    aggfunc=lambda x: len(x.unique()),
    margins=True,
    margins_name="Total"
).reset_index()

status_df

,status_report,affinity_name,offer_number,policy_number
0,Adaptedproposalwithdrawn,other,1,1
1,Calculatenewpremium,Insuro,348,1
2,Calculatenewpremium,Seguros International Ltd.,366,1
3,Calculatenewpremium,T&B,1255,1
4,Calculatenewpremium,other,909,1
5,Incompleterequest,Insuro,459,6
6,Incompleterequest,Seguros International Ltd.,524,11
7,Incompleterequest,T&B,1056,1
8,Incompleterequest,other,687,5
9,Personaloffer,Insuro,86,1


In [40]:
df.pivot_table(
    index=["status_report","affinity_name"],
    #columns="affinity_name",
    values=['offer_number','policy_number'],
    aggfunc=lambda x: len(x.unique()),
    margins=True,
    margins_name="Total"
).reset_index()

,status_report,affinity_name,offer_number,policy_number
0,Adaptedproposalwithdrawn,other,1,1
1,Calculatenewpremium,Insuro,348,1
2,Calculatenewpremium,Seguros International Ltd.,366,1
3,Calculatenewpremium,T&B,1255,1
4,Calculatenewpremium,other,909,1
5,Incompleterequest,Insuro,459,6
6,Incompleterequest,Seguros International Ltd.,524,11
7,Incompleterequest,T&B,1056,1
8,Incompleterequest,other,687,5
9,Personaloffer,Insuro,86,1


## EXCEL

In [41]:
def append_df_to_excel(df, excel_path, sheet_name, table_style=None):
    """
    Append *df* to *excel_path* in a new sheet called *sheet_name*.
    Creates the file if it doesn't exist yet.
    Optionally formats the range as an Excel 'Table' (striped style)  
    if you pass a *table_style* string, e.g. "TableStyleMedium9".
    """
    # Figure out whether the file already exists
    file_exists = os.path.exists(excel_path)

    # 1️⃣  Open writer in append ('a') or write ('w') mode
    with pd.ExcelWriter(
            excel_path,
            engine="openpyxl",
            mode="a" if file_exists else "w",
            # **NO** if_sheet_exists parameter → appends when the name is new
    ) as writer:
        if file_exists and sheet_name in writer.book.sheetnames:
            raise ValueError(f"'{sheet_name}' already exists! Pick another name.")

        # 2️⃣  Dump the DataFrame
        df.to_excel(writer, sheet_name=sheet_name, index=False)

        # 3️⃣  Optional: convert range to an Excel Table (nice stripes)
        if table_style:
            from openpyxl.worksheet.table import Table, TableStyleInfo

            ws = writer.book[sheet_name]
            max_row, max_col = ws.max_row, ws.max_column
            last_col = chr(ord("A") + max_col - 1)      # crude col-letter calc
            excel_range = f"A1:{last_col}{max_row}"

            tbl = Table(displayName=sheet_name.replace(" ", "_"), ref=excel_range)
            tbl.tableStyleInfo = TableStyleInfo(name=table_style,
                                                showRowStripes=True,
                                                showColumnStripes=False)
            ws.add_table(tbl)

In [43]:
sheets = {
    "1_Conv":    conv_table,      
    "2_Products": product_df,
    "3_Urban": urban_df, 
    "4_Stage": stage_df,
    "5_Income": income_df,
    "6_Residence": residence_df,
    "7_Brand": brand_df,
    "8_Age": age_df,
    "9_Loan": loan_df,
    "10_Savings": savings_df,
    "11_Car": car_df,
    "12_OwnHouse": own_house_df,
    "13_CarAge": car_age_df,
    "14_Status": status_df
}

# ── 2. Create / overwrite workbook with xlsxwriter ─────────────────────────
# NOTE: xlsxwriter **cannot** append to an existing .xlsx; it generates a
# brand-new file.  If Source_data.xlsx already exists, this call will replace
# it.  Include any other DataFrames in *this* block if you need them, too.
with pd.ExcelWriter(excel_path, engine="xlsxwriter") as writer:
    workbook = writer.book

    for sheet_name, df in sheets.items():
        # 2-a. Dump the DataFrame
        df.to_excel(writer, sheet_name=sheet_name, startrow=0, startcol=0,
                    index=False)

        # 2-b. Turn that range into a nicely styled Excel Table
        worksheet = writer.sheets[sheet_name]
        max_row, max_col = df.shape            # rows & cols in the DataFrame
        table_range = (0, 0, max_row, max_col-1)  # (first_row, first_col, ...)

        worksheet.add_table(*table_range, {
            "name":       sheet_name.replace(" ", "_"),
            "columns":    [{"header": col} for col in df.columns],
            "style":      "Table Style Medium 9",
        })

        # 2-c. Auto-fit the column widths (simple heuristic)
        for i, col in enumerate(df.columns):
            col_width = max(len(str(col)), 12)                # header width
            try:
                col_width = max(col_width, int(df[col].astype(str).str.len().max()))
            except ValueError:                                # empty col
                pass
            worksheet.set_column(i, i, col_width + 2)         # +2 padding

C:\Users\dimet\AppData\Local\Programs\Python\Python313\Lib\site-packages\xlsxwriter\worksheet.py:3303: UserWarning: Invalid Excel characters in add_table(): '1_Conv'
  warn(f"Invalid Excel characters in add_table(): '{name}'")
C:\Users\dimet\AppData\Local\Programs\Python\Python313\Lib\site-packages\xlsxwriter\worksheet.py:3303: UserWarning: Invalid Excel characters in add_table(): '2_Products'
  warn(f"Invalid Excel characters in add_table(): '{name}'")
C:\Users\dimet\AppData\Local\Programs\Python\Python313\Lib\site-packages\xlsxwriter\worksheet.py:3303: UserWarning: Invalid Excel characters in add_table(): '3_Urban'
  warn(f"Invalid Excel characters in add_table(): '{name}'")
C:\Users\dimet\AppData\Local\Programs\Python\Python313\Lib\site-packages\xlsxwriter\worksheet.py:3303: UserWarning: Invalid Excel characters in add_table(): '4_Stage'
  warn(f"Invalid Excel characters in add_table(): '{name}'")
C:\Users\dimet\AppData\Local\Programs\Python\Python313\Lib\site-packages\xlsxwriter\wo